# Disaggregation

In [ ]:
from __future__ import print_function, division
import time

from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

In [ ]:
rcParams['figure.figsize'] = (30, 6)

### Dividing data into train and test set

In [ ]:
train = DataSet('/home/eric/Project/data/redd.h5')
test = DataSet('/home/eric/Project/data/redd.h5')

Let us use building 1 for demo purposes

In [ ]:
building = 1

Let's split data at April 30th

In [ ]:
train.set_window(start="2011-04-21",end="2011-05-05")
test.set_window(start="2011-05-30")

train_elec = train.buildings[1].elec

test_elec = test.buildings[1].elec

### Visualizing the data

In [ ]:
train_elec.plot()

In [ ]:
test_elec.mains().plot()

REDD data set has got appliance level data sampled every 3 or 4 seconds and mains data sampled every 1 second. Let us verify the same.

In [ ]:
fridge_meter = train_elec['fridge']
microwave_meter = train_elec['microwave']

In [ ]:
fridge_df = next(fridge_meter.load())
microwave_df = next(microwave_meter.load())

In [ ]:
fridge_meter.plot()
microwave_meter.plot()

In [ ]:
fridge_df.head()

In [ ]:
mains = train_elec.mains()

In [ ]:
mains_df = next(mains.load())

In [ ]:
mains_df.head()

Since, both of these are sampled at different frequencies, we will downsample both to 1 minute resolution. We will also select the top-5 appliances in terms of energy consumption and use them for training our FHMM and CO models.

### Selecting top-5 appliances

In [ ]:
top_5_train_elec = train_elec.submeters().select_top_k(k=5)

In [ ]:
top_5_train_elec

### Training and disaggregation

#### A function to disaggregate the mains data to constituent appliances and return the predictions

In [ ]:
def predict(clf, test_elec, sample_period, timezone):
    pred = {}
    gt= {}
    
    # "ac_type" varies according to the dataset used. 
    # Make sure to use the correct ac_type before using the default parameters in this code.    
    for i, chunk in enumerate(test_elec.mains().load(physical_quantity = 'power', ac_type = 'apparent', sample_period=sample_period)):
        chunk_drop_na = chunk.dropna()
        pred[i] = clf.disaggregate_chunk(chunk_drop_na)
        gt[i]={}

        for meter in test_elec.submeters().meters:
            # Only use the meters that we trained on (this saves time!)    
            gt[i][meter] = next(meter.load(physical_quantity = 'power', ac_type = 'active', sample_period=sample_period))
        gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i]) if len(v)}, index=next(iter(gt[i].values())).index).dropna()
        
    # If everything can fit in memory
    gt_overall = pd.concat(gt)
    gt_overall.index = gt_overall.index.droplevel()
    pred_overall = pd.concat(pred)
    pred_overall.index = pred_overall.index.droplevel()

    # Having the same order of columns
    gt_overall = gt_overall[pred_overall.columns]
    
    #Intersection of index
    gt_index_utc = gt_overall.index.tz_convert("UTC")
    pred_index_utc = pred_overall.index.tz_convert("UTC")
    common_index_utc = gt_index_utc.intersection(pred_index_utc)
    
    common_index_local = common_index_utc.tz_convert(timezone)
    gt_overall = gt_overall.loc[common_index_local]
    pred_overall = pred_overall.loc[common_index_local]
    appliance_labels = [m for m in gt_overall.columns.values]
    gt_overall.columns = appliance_labels
    pred_overall.columns = appliance_labels
    return gt_overall, pred_overall

#### Train using 2 benchmarking algorithms -  Combinatorial Optimisation (CO) and Factorial Hidden Markov Model (FHMM)

In [ ]:
classifiers = {'CO':CombinatorialOptimisation(), 'FHMM':FHMM()}
predictions = {}
sample_period = 120
for clf_name, clf in classifiers.items():
    print("*"*20)
    print(clf_name)
    print("*" *20)
    start = time.time()
    # Note that we have given the sample period to downsample the data to 1 minute. 
    # If instead of top_5 we wanted to train on all appliance, we would write 
    # fhmm.train(train_elec, sample_period=60)
    clf.train(top_5_train_elec, sample_period=sample_period)
    end = time.time()
    print("Runtime =", end-start, "seconds.")
    gt, predictions[clf_name] = predict(clf, test_elec, sample_period, train.metadata['timezone'])
   

Using prettier labels!

In [ ]:
appliance_labels = [m.label() for m in gt.columns.values]

In [ ]:
gt.columns = appliance_labels
predictions['CO'].columns = appliance_labels
predictions['FHMM'].columns = appliance_labels

#### Taking a look at the ground truth of top 5 appliance power consumption

In [ ]:
gt.head()

In [ ]:
predictions['CO'].head()

In [ ]:
predictions['FHMM'].head()

### Plotting the predictions against the actual usage

In [ ]:
predictions['CO']['Fridge'].head(300).plot(label="Pred")
gt['Fridge'].head(300).plot(label="GT")
plt.legend()

In [ ]:
predictions['FHMM']['Fridge'].head(300).plot(label="Pred")
gt['Fridge'].head(300).plot(label="GT")
plt.legend()

### Comparing NILM algorithms (CO vs FHMM)

`nilmtk.utils.compute_rmse` is an extended of the following, handling both missing values and labels better:
```python
def compute_rmse(gt, pred):
    from sklearn.metrics import mean_squared_error
    rms_error = {}
    for appliance in gt.columns:
        rms_error[appliance] = np.sqrt(mean_squared_error(gt[appliance], pred[appliance]))
    return pd.Series(rms_error)
```

In [ ]:
? nilmtk.utils.compute_rmse

In [ ]:
rmse = {}
for clf_name in classifiers.keys():
    rmse[clf_name] = nilmtk.utils.compute_rmse(gt, predictions[clf_name])

rmse = pd.DataFrame(rmse)
rmse